### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 7

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Коняева Марина Александровна
##### Группа:   НФИбд-01-21

## Москва 2024
***

## Вариант № 4






Регрессоры: 

⁃               Полиномиальная регрессия (PolynomialFeatures+LinearRegression), параметр degree в диапазоне от 2 до 5 

⁃               гребневая регрессия (Ridge), параметр alpha в диапазоне от 0.1 до 1 

⁃               регрессии на основе метода опорных векторов (SVR), параметр degree в диапазоне от 1 до 5 

Показатели качества регрессии: 

⁃               Для определения лучшего регрессора MSPE 

⁃               Для визуализации кривой обучения MSLE 

1. Cчитайте заданный набор данных из репозитария UCI, включая указанный в индивидуальном задании столбец с метками классов и столбец с откликом (зависимой переменной).

Automobile Data Set 

Название файла: imports-85.data 

Класс: symboling (столбец No 1)

Зависимая переменная: price (столбец No 26) 

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [62]:
# считываем данные из репозитария UCI
url = 'imports-85.data'
my_data = pd.read_csv(url, header=None)
my_data = my_data.rename(columns={0: "class"})
my_data = my_data.rename(columns={25: "price"})
my_data

,class,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


2. Преобразуйте в числовые признаки неправильно распознанные признаки с числовыми значениями. Если в столбцах с метками классов и откликом имеются пропущенные значения, то удалите записи с пропущенными значениями. Оставьте в наборе данных только числовые признаки.

In [63]:
# среди меток класса отсуствуют пропущенные значения
my_data['class'].isnull().sum(axis=0)

0

In [64]:
# пропущенных значений в откликах не обнаружено 
my_data['price'].isnull().sum(axis=0)

0

In [65]:
# проверим категориальные признаки
my_data.dtypes

class      int64
1         object
2         object
3         object
4         object
5         object
6         object
7         object
8         object
9        float64
10       float64
11       float64
12       float64
13         int64
14        object
15        object
16         int64
17        object
18        object
19        object
20       float64
21        object
22        object
23         int64
24         int64
price     object
dtype: object

In [66]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {} # экземпляр LabelEncoder для каждого категориального столбца
categorical_columns = [1,2,3,4,5,6,7,8,14,15,17,18,19,21,22,'price']

for col in categorical_columns:
    le = LabelEncoder()
    my_data[col] = le.fit_transform(my_data[col])
    label_encoders[col] = le

In [67]:
# теперь категориальные значения закодированы числовыми значениями
my_data.dtypes

class      int64
1          int32
2          int32
3          int32
4          int32
5          int32
6          int32
7          int32
8          int32
9        float64
10       float64
11       float64
12       float64
13         int64
14         int32
15         int32
16         int64
17         int32
18         int32
19         int32
20       float64
21         int32
22         int32
23         int64
24         int64
price      int32
dtype: object

In [68]:
my_data

,class,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,price
0,3,51,0,1,0,2,0,2,0,88.6,...,130,5,23,4,9.0,5,10,21,27,31
1,3,51,0,1,0,2,0,2,0,88.6,...,130,5,23,4,9.0,5,10,21,27,50
2,1,51,0,1,0,2,2,2,0,94.5,...,152,5,1,27,9.0,20,10,19,26,50
3,2,27,1,1,0,1,3,1,0,99.8,...,109,5,13,24,10.0,2,16,24,30,36
4,2,27,1,1,0,1,3,0,0,99.4,...,136,5,13,24,8.0,8,16,18,22,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,49,21,1,0,1,3,2,0,109.1,...,141,5,35,15,9.5,7,15,23,28,56
201,-1,49,21,1,1,1,3,2,0,109.1,...,141,5,35,15,8.7,23,14,19,25,73
202,-1,49,21,1,0,1,3,2,0,109.1,...,173,5,26,7,8.8,13,16,18,23,77
203,-1,49,21,0,1,1,3,2,0,109.1,...,145,3,6,24,23.0,3,8,26,27,79


3. Если в наборе данных остались пропущенные значения, то замените пропущенные значения, используя метод, указанный в индивидуальном задании. Если пропущенные значения в наборе данных отсутствуют, то определите и удалите точки с выбросами в соответствии с методом, указанным в индивидуальном задании. Выберите параметры методов таким образом, чтобы выбросы составляли не менее 5% всех точек набора данных.

In [69]:
# пропущенные значения в наборе данных отсуствуют
missing_values = my_data.isna().sum()
missing_values

class    0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
price    0
dtype: int64

Метод обработки выбросов – алгоритм кластеризации OPTICS 

In [70]:
from sklearn.cluster import DBSCAN

my_data.columns = my_data.columns.astype(str)

# параметры DBSCAN
eps = 1.0  # Радиус эпсилон
min_samples = 1  # Минимальное количество точек в кластере

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(my_data)
# подсчет количества выбросов
outliers_count = len(labels[labels == -1])
# подсчет общего количества точек в наборе данных
total_points = len(my_data)
# рассчет процент выбросов
outliers_percentage = (outliers_count / total_points) * 100

print(f"Процент выбросов: {outliers_percentage}%")

# удаление выбросов
my_data = my_data[labels != -1]

Процент выбросов: 0.0%


In [71]:
import pandas as pd
from sklearn.cluster import OPTICS

# ... (ваш предыдущий код)

# Параметры OPTICS
min_samples = 1 # Минимальное количество точек в кластере
max_eps = 1.0 # Максимальный радиус эпсилон (для определения плотности)
xi = 0.05 # Шаг изменения радиуса эпсилон

# Поиск параметров, дающих не менее 5% выбросов
while True:
  optics = OPTICS(min_samples=min_samples, max_eps=max_eps, xi=xi)
  labels = optics.fit_predict(my_data)
  outliers_count = len(labels[labels == -1])
  outliers_percentage = (outliers_count / len(my_data)) * 100
  print(f"Процент выбросов: {outliers_percentage}%")

  if outliers_percentage >= 5:
    break
  else:
    # Уменьшаем max_eps, чтобы увеличить количество выбросов
    max_eps *= 0.8

# Удаление выбросов
my_data = my_data[labels != -1]


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


...

Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%
Процент выбросов: 100.0%


KeyboardInterrupt: 

In [59]:
my_data = my_data.reset_index(drop=True)
my_data

,class,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,price


4. Масштабируйте признаки набора данных на интервал [0, 1]. Используя метод снижения размерности данных, указанный в индивидуальном задании, оставьте в наборе данных три признака (кроме метки класса и откликов), принимающих более 50 различных значений.

Метод снижения размерности данных – линейная регрессия (LinearRegression) 

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)

In [ ]:
abs(lin_reg.coef_)

In [ ]:
coefficients = np.array(abs(lin_reg.coef_))
sorted_indices = np.argsort(coefficients)
top_3_indices = sorted_indices[-3:]
top_3_values = coefficients[top_3_indices]
top_3_values

In [ ]:
top_3_indices

In [ ]:
my_data3 = my_new_data[[2,22,25,'price]].join(my_data['class'])
my_data3

5. Визуализируйте набор данных в виде точек в трехмерном пространстве, отображая точки разных классов разными цветами. В качестве подписей осей используйте названия признаков. В подписи рисунка укажите название набора данных. Создайте легенду набора данных.

6. Разбейте набор данных на обучающую и тестовую выборки. Постройте регрессоры на базе моделей регрессии, указанных в индивидуальном задании, для каждого из трех признаков. Определите оптимальные параметры регрессоров при помощи GridSearchCV.

7. Для каждого из трех признаков визуализируйте на плоскости набор данных одним цветом и линии регрессии для регрессоров с оптимальными параметрами, определенными в п. 6 (всего три рисунка). Регрессоры, имеющие максимальное значение показателя качества регрессии, указанного в индивидуальном задании, выделите красным цветом. В качестве подписи оси X используйте название признака, в качестве подписи оси Y – название столбца с откликами. Создайте легенду для линий регрессии.

8. Постройте на одном рисунке кривые обучения (зависимость показателя качества регрессии, указанного в индивидуальном задании, от количества точек в обучающей выборке) для трех лучших регрессоров для каждого из трех признаков по показателю качества, указанному в индивидуальном задании. Кривые для регрессора с максимальным показателем качества визуализируйте красным цветом (кривую для обучающей выборки сплошной линией, кривую для тестовой выборки линией из точек). Подпишите корректно оси и создайте легенду для кривых обучения.